In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/2f/9101t8nx1hn47xc4fzmkb13w0000gn/T/ipykernel_32117/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [14]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)
print(data.head(100))
print(data.dtypes)

(1000, 2)
                                                 text  label
0   DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                            Will do.      0
2   Nora--Cheryl has emailed dozens of memos about...      0
3   Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                 fyi      0
..                                                ...    ...
95  Subject to your satisfaction you will be given...      1
96                    Also I should mention the G-20.      0
97  RGVhciBmcmllbmQsIAoKR3JlZXRpbmdzIHRvIHlvdSBpbi...      1
98  cheryl.mills B6Sunday August 2 2009 8:50 AMHRe...      0
99  Thanks for the help. We're working to schedule...      0

[100 rows x 2 columns]
text     object
label     int64
dtype: object


### Let's divide the training and test set into two partitions

In [15]:
# Your code
from sklearn.model_selection import train_test_split

# Separate features and target
X = data['text']
y = data['label']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split
print("Training set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])

Training set size: 800
Validation set size: 200


## Data Preprocessing

In [16]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [18]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [19]:
# Your code
import re
from bs4 import BeautifulSoup

def clean_html(text):
    # Step 1: Remove inline JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    
    # Step 2: Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Step 3: Remove remaining HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    return text

# Apply the cleaning function to the text column
data['clean_text'] = data['text'].apply(clean_html)

# Display the first few rows of the cleaned text
print(data[['text', 'clean_text']].head())

                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                          clean_text  
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...  
1                                           Will do.  
2  Nora--Cheryl has emailed dozens of memos about...  
3  Dear Sir=2FMadam=2C I know that this proposal ...  
4                                                fyi  


/var/folders/2f/9101t8nx1hn47xc4fzmkb13w0000gn/T/ipykernel_32117/1516230239.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()
/var/folders/2f/9101t8nx1hn47xc4fzmkb13w0000gn/T/ipykernel_32117/1516230239.py:14: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [20]:
# Your code
import re

def clean_text(text):
    # Remove all special characters
    text = re.sub(r'\W', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'\b\w\b\s?', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Apply the cleaning function to the clean_text column
data['cleaned_text'] = data['clean_text'].apply(clean_text)

# Display the first few rows of the cleaned text
print(data[['text', 'clean_text', 'cleaned_text']].head())

                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                          clean_text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strictly private business proposal am...  
1                                           will do   
2  nora cheryl has emailed dozens of memos about ...  
3  dear sir fmadam know that this proposal might ...  
4                                                fy

## Now let's work on removing stopwords
Remove the stopwords.

In [21]:
# Your code
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Load stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # Split the text into words
    words = text.split()
    
    # Remove stopwords
    filtered_words = [word for word in words if word not in stop_words]
    
    # Join the filtered words back into a single string
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

# Apply the function to remove stopwords from the cleaned_text column
data['final_cleaned_text'] = data['cleaned_text'].apply(remove_stopwords)

# Display the first few rows of the final cleaned text
print(data[['text', 'cleaned_text', 'final_cleaned_text']].head())

                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  \
0  dear sir strictly private business proposal am...   
1                                           will do    
2  nora cheryl has emailed dozens of memos about ...   
3  dear sir fmadam know that this proposal might ...   
4                                                fyi   

                                  final_cleaned_text  
0  dear sir strictly private business proposal mi...  
1                                                     
2  nora cheryl emailed dozens memos haiti weekend...  
3  dear sir fmadam know proposal might surprise e...  
4                                                fy

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [22]:
# Your code
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Ensure you have the required NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join the lemmatized words back into a single string
    lemmatized_text = ' '.join(lemmatized_words)
    
    return lemmatized_text

# Apply the lemmatization function to the final_cleaned_text column
data['lemmatized_text'] = data['final_cleaned_text'].apply(lemmatize_text)

# Display the first few rows of the lemmatized text
print(data[['text', 'final_cleaned_text', 'lemmatized_text']].head())

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cindymartinezgrullon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cindymartinezgrullon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                  final_cleaned_text  \
0  dear sir strictly private business proposal mi...   
1                                                      
2  nora cheryl emailed dozens memos haiti weekend...   
3  dear sir fmadam know proposal might surprise e...   
4                                                fyi   

                                     lemmatized_text  
0  dear sir strictly private business proposal mi...  
1                                                     
2  nora cheryl emailed dozen memo haiti weekend p...  
3  dear sir fmadam know proposal might surprise e...  
4                                                fy

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Remove empty documents after preprocessing
data['final_cleaned_text'] = data['final_cleaned_text'].replace('', None)
data = data.dropna(subset=['final_cleaned_text'])

# Separate ham and spam messages
ham_messages = data[data['label'] == 0]['final_cleaned_text']
spam_messages = data[data['label'] == 1]['final_cleaned_text']

# Create a CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform ham messages
ham_bow = vectorizer.fit_transform(ham_messages)
ham_words = vectorizer.get_feature_names_out()
ham_counts = ham_bow.sum(axis=0).A1
ham_word_counts = dict(zip(ham_words, ham_counts))

# Fit and transform spam messages
spam_bow = vectorizer.fit_transform(spam_messages)
spam_words = vectorizer.get_feature_names_out()
spam_counts = spam_bow.sum(axis=0).A1
spam_word_counts = dict(zip(spam_words, spam_counts))

# Get top 10 words in ham and spam messages
top_ham_words = sorted(ham_word_counts.items(), key=lambda item: item[1], reverse=True)[:10]
top_spam_words = sorted(spam_word_counts.items(), key=lambda item: item[1], reverse=True)[:10]

# Create DataFrame to display results
top_words_df = pd.DataFrame({
    'Ham Words': [word[0] for word in top_ham_words],
    'Ham Counts': [word[1] for word in top_ham_words],
    'Spam Words': [word[0] for word in top_spam_words],
    'Spam Counts': [word[1] for word in top_spam_words]
})

print(top_words_df)

## Extra features

In [25]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).

money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

# Add indicators and text length to the dataset
data['money_mark'] = data['lemmatized_text'].str.contains(money_simbol_list, case=False)*1
data['suspicious_words'] = data['lemmatized_text'].str.contains(suspicious_words, case=False)*1
data['text_len'] = data['lemmatized_text'].apply(lambda x: len(x))

# Display the first few rows of the dataset with new columns
print(data[['text', 'lemmatized_text', 'money_mark', 'suspicious_words', 'text_len']].head())

                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                     lemmatized_text  money_mark  \
0  dear sir strictly private business proposal mi...           1   
1                                                              1   
2  nora cheryl emailed dozen memo haiti weekend p...           1   
3  dear sir fmadam know proposal might surprise e...           1   
4                                                fyi           1   

   suspicious_words  text_len  
0                 1      1494  
1                 0         0  
2                 0       111  
3                 1      1346  
4                 0         3  


## How would work the Bag of Words with Count Vectorizer concept?

In [26]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Sample text data
documents = [
    "I love programming in Python",
    "Python is a great programming language",
    "I also enjoy working with data"
]

# Create a CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the documents
X = vectorizer.fit_transform(documents)

# Convert the result to a DataFrame for better visualization
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

print(bow_df)

   also  data  enjoy  great  in  is  language  love  programming  python  \
0     0     0      0      0   1   0         0     1            1       1   
1     0     0      0      1   0   1         1     0            1       1   
2     1     1      1      0   0   0         0     0            0       0   

   with  working  
0     0        0  
1     0        0  
2     1        1  


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [27]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Vectorize the entire dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(data['lemmatized_text'])

# Print the shape of the vectorized dataset
print(tfidf_matrix.shape)

(1000, 22016)


## And the Train a Classifier?

In [28]:
# Your code
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['label'], test_size=0.2, random_state=42)

# Initialize the Logistic Regression classifier
classifier = LogisticRegression()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=1)
recall = recall_score(y_test, y_pred, pos_label=1)
f1 = f1_score(y_test, y_pred, pos_label=1)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

Accuracy: 0.94
Precision: 1.00
Recall: 0.85
F1 Score: 0.92


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).